### BOW Vs TFIDF

In [1]:
import dagshub
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/aryan0147/Capstone-Project-2.mlflow")

dagshub.init(repo_owner='aryan0147', repo_name='Capstone-Project-2', mlflow=True)

Accessing as aryan0147

Initialized MLflow to track repo "aryan0147/Capstone-Project-2"

Repository aryan0147/Capstone-Project-2 initialized!

In [4]:
# Set or create an experiment
mlflow.set_experiment("Exp 1-BoW vs TfIdf")

2025/02/10 19:07:18 INFO mlflow.tracking.fluent: Experiment with name 'Exp 1-BoW vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/665f9e0419cb44debd1e994c132da95a', creation_time=1739194640710, experiment_id='8', last_update_time=1739194640710, lifecycle_stage='active', name='Exp 1-BoW vs TfIdf', tags={}>

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [6]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [7]:
import mlflow
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Train-test split before any preprocessing (Prevents Data Leakage)
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
)

# Step 2: Function to vectorize data
def vectorize_data(vectorizer_type, ngram_range, max_features):
    """Applies vectorization only on training data and transforms test data accordingly."""
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train_vec = vectorizer.fit_transform(X_train)  # Fit only on training data
    X_test_vec = vectorizer.transform(X_test)  # Transform test data without refitting

    return X_train_vec, X_test_vec, vectorizer

# Step 3: Function to run the experiment
def run_experiment(vectorizer_type, ngram_range, max_features, vectorizer_name):
    """Trains and logs a Random Forest model with MLflow, ensuring no data leakage."""
    
    # Vectorize the data
    X_train_vec, X_test_vec, vectorizer = vectorize_data(vectorizer_type, ngram_range, max_features)

    # Define RandomForest model parameters
    n_estimators = 200
    max_depth = 15

    # Train model inside MLflow tracking
    with mlflow.start_run():
        # Log experiment details
        mlflow.set_tag("mlflow.source.name", "experiment_1_bow_tfidf.ipynb")
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest hyperparameters
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train_vec, y_train)

        # Make predictions
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix safely
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the trained model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# Step 4: Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # Unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")


2025/02/10 19:08:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/8/runs/34c1310e955643ee87f9ef9d3c037aba
🧪 View experiment at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/8


2025/02/10 19:09:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/8/runs/a737eeac985b454e8e0cc1f6fdf58d84
🧪 View experiment at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/8


2025/02/10 19:09:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/8/runs/044cf639debb4952bcc2e69a470aa837
🧪 View experiment at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/8


2025/02/10 19:11:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/8/runs/8711bab3f4f2454f939f2467b02995aa
🧪 View experiment at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/8


2025/02/10 19:11:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/8/runs/fd6d0a34b94d45d7bf72320db8cd8f0b
🧪 View experiment at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/8
